Below is an ETL pipeline to visualize, understand, and clean the "DOHMH-New-York-City-Restaurant-Inspection-Results" dataset. The cleaned dataset it written to 'cleaned_data.csv'.

In [1]:
import pandas as pd


In [2]:
# Read in the CSV file to a pandas dataframe
health_df = pd.read_csv('DOHMH_New_York_City_Restaurant_Inspection_Results.csv')


/Users/Conor/anaconda2/lib/python2.7/site-packages/IPython/core/interactiveshell.py:2714: DtypeWarning: Columns (6) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


**CAMIS** | This is an unique identifier for the entity (restaurant); 10-digit integer, static per restaurant permit | *Plain Text*

**DBA** | This field represents the name (doing business as) of the entity (restaurant); Public business name, may change at discretion of restaurant owner | *Plain Text*

**BORO** | Borough in which the entity (restaurant) is located.;• 1 = MANHATTAN • 2 = BRONX • 3 = BROOKLYN • 4 = QUEENS • 5 = STATEN ISLAND • Missing; NOTE: There may be discrepancies between zip code and listed boro due to differences in an establishment's mailing address and physical location | *Plain Text*

**BUILDING** | Building number for establishment (restaurant) location | *Plain Text*

**STREET** | Street name for establishment (restaurant) location | *Plain Text*

**ZIPCODE** | Zip code of establishment (restaurant) location | *Plain Text*

**PHONE** | Phone Number; Phone number provided by restaurant owner/manager | *Plain Text*
 
**CUISINE DESCRIPTION** | This field describes the entity (restaurant) cuisine. ; Optional field provided by provided by restaurant owner/manager | *Plain Text*

**INSPECTION DATE** | This field represents the date of inspection; NOTE: Inspection dates of 1/1/1900 mean an establishment has not yet had an inspection | *Date & Time*

**ACTION** | This field represents the actions that is associated with each restaurant inspection. ; • Violations were cited in the following area(s). • No violations were recorded at the time of this inspection. • Establishment re-opened by DOHMH • Establishment re-closed by DOHMH • Establishment Closed by DOHMH. Violations were cited in the following area(s) and those requiring immediate action were addressed. • "Missing" = not yet inspected; | *Plain Text*

**VIOLATION CODE** | Violation code associated with an establishment (restaurant) inspection | *Plain Text*

**VIOLATION DESCRIPTION** | Violation description associated with an establishment (restaurant) inspection | *Plain Text*

**CRITICAL FLAG** | Indicator of critical violation; "• Critical • Not Critical • Not Applicable"; Critical violations are those most likely to contribute to food-borne illness | *Plain Text*

**SCORE** | Total score for a particular inspection; Scores are updated based on adjudication results | *Number*

**GRADE** | Grade associated with the inspection; • N = Not Yet Graded• A = Grade A• B = Grade B• C = Grade C• Z = Grade Pending• P= Grade Pending issued on re-opening following an initial inspection that resulted in a closure | *Plain Text*

**GRADE DATE** | The date when the current grade was issued to the entity (restaurant) | *Date & Time*

**RECORD DATE** | The date when the extract was run to produce this data set | *Date & Time*

**INSPECTION TYPE** | A combination of the inspection program and the type of inspection performed; See Data Dictionary for full list of expected values | *Plain Text*



In [3]:
# Since we are reading in a static file, we can assert known attributes for sanity testing

# Comment out because this is limiting for new data...
# assert(health_df.shape == (384391, 18))
# assert(health_df.size == 6919038)


In [4]:
# Print the data for reference
health_df.head()


,CAMIS,DBA,BORO,BUILDING,STREET,ZIPCODE,PHONE,CUISINE DESCRIPTION,INSPECTION DATE,ACTION,VIOLATION CODE,VIOLATION DESCRIPTION,CRITICAL FLAG,SCORE,GRADE,GRADE DATE,RECORD DATE,INSPECTION TYPE
0,30075445,MORRIS PARK BAKE SHOP,BRONX,1007,MORRIS PARK AVE,10462.0,7188924968,Bakery,06/11/2019,Violations were cited in the following area(s).,08C,Pesticide use not in accordance with label or ...,Not Critical,6.0,A,06/11/2019,07/17/2019,Cycle Inspection / Re-inspection
1,30075445,MORRIS PARK BAKE SHOP,BRONX,1007,MORRIS PARK AVE,10462.0,7188924968,Bakery,06/11/2019,Violations were cited in the following area(s).,10B,Plumbing not properly installed or maintained;...,Not Critical,6.0,A,06/11/2019,07/17/2019,Cycle Inspection / Re-inspection
2,30075445,MORRIS PARK BAKE SHOP,BRONX,1007,MORRIS PARK AVE,10462.0,7188924968,Bakery,06/11/2019,Violations were cited in the following area(s).,10F,Non-food contact surface improperly constructe...,Not Critical,6.0,A,06/11/2019,07/17/2019,Cycle Inspection / Re-inspection
3,30075445,MORRIS PARK BAKE SHOP,BRONX,1007,MORRIS PARK AVE,10462.0,7188924968,Bakery,05/16/2019,Violations were cited in the following area(s).,04L,Evidence of mice or live mice present in facil...,Critical,14.0,NaN,NaN,07/17/2019,Cycle Inspection / Initial Inspection
4,30075445,MORRIS PARK BAKE SHOP,BRONX,1007,MORRIS PARK AVE,10462.0,7188924968,Bakery,05/16/2019,Violations were cited in the following area(s).,08A,Facility not vermin proof. Harborage or condit...,Not Critical,14.0,NaN,NaN,07/17/2019,Cycle Inspection / Initial Inspection


In [5]:
# Display unique values from each category
for cat in health_df.columns.values:
    print(cat, health_df[cat].unique())

# Looks like we will have to clean up CUISINE DESCRIPTION...


('CAMIS', array([30075445, 30112340, 30191841, ..., 50095591, 50095592, 50095596]))
('DBA', array(['MORRIS PARK BAKE SHOP', "WENDY'S",
       'DJ REYNOLDS PUB AND RESTAURANT', ..., 'TULO HOUSE', 'ROSATORO',
       'WAYO, BAR WAYO, WAYO YAKINIKU'], dtype=object))
('BORO', array(['BRONX', 'BROOKLYN', 'MANHATTAN', 'QUEENS', 'STATEN ISLAND',
       'Missing'], dtype=object))
('BUILDING', array(['1007', '469', '351', ..., '9415', '25211', '18712'], dtype=object))
('STREET', array(['MORRIS PARK AVE', 'FLATBUSH AVENUE', 'WEST   57 STREET', ...,
       'WASHINGTON SQ W', 'OAKPOINT AVE', 'E 222ND ST'], dtype=object))
('ZIPCODE', array([10462., 11225., 10019., 11224., 11369., 11234., 11226., 10460.,
       10065., 11223., 11218., 11004., 11356., 11106., 10025., 10012.,
       11368., 10005., 11203., 11201., 10018., 10174., 10314., 11379.,
       10028., 10022., 10305., 11212., 10023.,    nan, 11230., 10306.,
       10458., 10308., 11209., 11211., 10465., 10003., 11222., 10014.,
       10013., 11

      dtype=object))


In [6]:
# Get type of each column 
health_df.info()  


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 384391 entries, 0 to 384390
Data columns (total 18 columns):
CAMIS                    384391 non-null int64
DBA                      383837 non-null object
BORO                     384391 non-null object
BUILDING                 384144 non-null object
STREET                   384388 non-null object
ZIPCODE                  378937 non-null float64
PHONE                    384374 non-null object
CUISINE DESCRIPTION      384391 non-null object
INSPECTION DATE          384391 non-null object
ACTION                   383063 non-null object
VIOLATION CODE           378668 non-null object
VIOLATION DESCRIPTION    375494 non-null object
CRITICAL FLAG            384391 non-null object
SCORE                    367128 non-null float64
GRADE                    195317 non-null object
GRADE DATE               193179 non-null object
RECORD DATE              384391 non-null object
INSPECTION TYPE          383063 non-null object
dtypes: float64(2), int6

In [7]:
# Calculate percentage of missing data per column 
(health_df.isnull().sum() / float(len(health_df))) * 100

# Lots of missing GRADE's and GRADE_DATE's


CAMIS                     0.000000
DBA                       0.144124
BORO                      0.000000
BUILDING                  0.064257
STREET                    0.000780
ZIPCODE                   1.418868
PHONE                     0.004423
CUISINE DESCRIPTION       0.000000
INSPECTION DATE           0.000000
ACTION                    0.345482
VIOLATION CODE            1.488849
VIOLATION DESCRIPTION     2.314570
CRITICAL FLAG             0.000000
SCORE                     4.491000
GRADE                    49.187936
GRADE DATE               49.744141
RECORD DATE               0.000000
INSPECTION TYPE           0.345482
dtype: float64

We are prepping this data to store it in a sqlite database table.
The table will be general, i.e. an "establishments" table. Each table
entry will contain the basic establishments' info shown below. It can be considered a snapshot of each restaurant based on the most recent data we have. 

For all entries, we want:
- Name (DBA)
- Address (BORO, BUILDING, STREET, ZIPCODE)
- Phone (PHONE)
- Cuisine description (CUISINE DESCRIPTION)
- Inspection grade (GRADE)
- If establishment has been inspected (INSPECTION DATE)
- If establishment has been closed based on inspection (ACTION)


In [8]:
# Each restaurant listed can have one or more inspections. For our table, we only want one entry per restaurant. 

# Sort by INSPECTION TYPE
health_df.sort_values(by=['INSPECTION DATE'])

# Drop DBA for older inspections (keeps the first instance of each)
health_df.drop_duplicates(subset =['DBA'], inplace = True)


In [9]:
# We can start dropping columns that won't be included in the database.

# We are not using the CAMIS ID to join with another dataset - drop.
health_df.drop(['CAMIS'], axis = 1, inplace = True)
health_df.drop(['VIOLATION CODE'], axis = 1, inplace = True)
health_df.drop(['VIOLATION DESCRIPTION'], axis = 1, inplace = True)
health_df.drop(['CRITICAL FLAG'], axis = 1, inplace = True)
health_df.drop(['SCORE'], axis = 1, inplace = True)
health_df.drop(['GRADE DATE'], axis = 1, inplace = True)
health_df.drop(['RECORD DATE'], axis = 1, inplace = True)
health_df.drop(['INSPECTION TYPE'], axis = 1, inplace = True)

# Sanity check 
assert(health_df.shape[1] == 10)


In [10]:
null_DBA_rows = len(health_df) - health_df['DBA'].count()

# Establishments without names are useless - drop. 
health_df.dropna(subset = ['DBA'], inplace = True)

# Sanity check
assert(health_df.shape[1] == 10)


In [11]:
# We can use the ACTION column to determine if an establishment has been closed. 
closed_status = ['Establishment re-closed by DOHMH', 'Establishment Closed by DOHMH.  Violations were cited in the following area(s) and those requiring immediate action were addressed.']

is_closed = [True if action in closed_status else False for action in health_df['ACTION']]

health_df['CLOSED'] = is_closed 

# We no longer need this column - drop.
health_df.drop(['ACTION'], axis = 1, inplace = True)

# Sanity check 
assert(health_df.shape[1] == 10)


In [12]:
# Use the INSPECTION DATE to determine if an establishment has been inspected or not. 
not_inspected_status = '1/1/1900'

is_inspected = [False if date == not_inspected_status else True for date in health_df['INSPECTION DATE']]

health_df['INSPECTED'] = is_inspected 

# We no longer need this column - drop.
health_df.drop(['INSPECTION DATE'], axis = 1, inplace = True)

# Sanity check 
assert(health_df.shape[1] == 10)


In [13]:
# Clean the GRADE column.
# Values found in the GRADE column: 'A', nan, 'B', 'C', 'Z', 'P', 'Not Yet Graded', 'G'
# Values listed in the dictionary: 'A', 'B', 'C', 'Z', 'P', 'N'
# Replace instances of 'Not Yet Graded' with 'N'
clean_grades = ['N' if grade == 'Not Yet Graded' else grade for grade in health_df['GRADE']]

health_df['GRADE'] = clean_grades 

# Replace null entries with "U" - Unknown
health_df['GRADE'].fillna(value = 'U', inplace = True)

# Drop entries with grades not found in the dictionary
grade_types = ['A', 'B', 'C', 'Z', 'P', 'N', 'U']
health_df = health_df[health_df['GRADE'].isin(grade_types)]

# Sanity check 
assert(health_df.shape[1] == 10)


In [14]:
# Update the CUISINE DESCRIPTION column.
# There is one malformed entry that needs to be addressed. 
# This should further be cleaned so that cuisines are one word. 

# Bad entry - Caf\xc3\x83\xc2\xa9/Coffee/Tea
clean_cafes = ['cafe' if cuisine == 'Caf\xc3\x83\xc2\xa9/Coffee/Tea' else cuisine for cuisine in health_df['CUISINE DESCRIPTION']]

health_df['CUISINE DESCRIPTION'] = clean_cafes 

# Transform all types to lowercase. 
clean_cuisines = [cuisine.lower() for cuisine in health_df['CUISINE DESCRIPTION']]

health_df['CUISINE DESCRIPTION'] = clean_cuisines 

health_df.rename(columns={'CUISINE DESCRIPTION': 'CUISINE_DESCRIPTION'}, inplace = True)

# Sanity check 
assert(health_df.shape[1] == 10)


In [15]:
# Calculate percentage of missing data per column 
(health_df.isnull().sum() / float(len(health_df))) * 100


DBA                    0.000000
BORO                   0.000000
BUILDING               0.095039
STREET                 0.000000
ZIPCODE                1.610910
PHONE                  0.019008
CUISINE_DESCRIPTION    0.000000
GRADE                  0.000000
CLOSED                 0.000000
INSPECTED              0.000000
dtype: float64

In [16]:
# Display unique values from each category
for cat in health_df.columns.values:
    print(cat, health_df[cat].unique())


('DBA', array(['MORRIS PARK BAKE SHOP', "WENDY'S",
       'DJ REYNOLDS PUB AND RESTAURANT', ..., 'TULO HOUSE', 'ROSATORO',
       'WAYO, BAR WAYO, WAYO YAKINIKU'], dtype=object))
('BORO', array(['BRONX', 'BROOKLYN', 'MANHATTAN', 'QUEENS', 'STATEN ISLAND',
       'Missing'], dtype=object))
('BUILDING', array(['1007', '469', '351', ..., '10222A', '2234', '693'], dtype=object))
('STREET', array(['MORRIS PARK AVE', 'FLATBUSH AVENUE', 'WEST   57 STREET', ...,
       'E 206TH ST', 'W 167TH ST', 'BRIDGE PARK DR'], dtype=object))
('ZIPCODE', array([10462., 11225., 10019., 11224., 11369., 11234., 11226., 10460.,
       10065., 11223., 11218., 11356., 11106., 10025., 10012., 11368.,
       10005., 11203., 11201., 10018., 10174., 10314., 11379., 10028.,
       10022., 11212., 10023.,    nan, 11230., 10306., 10458., 10308.,
       11209., 11211., 10465., 10003., 11222., 10014., 10013., 11385.,
       10009., 10016., 11235., 10011., 11414., 11363., 11103., 10305.,
       10309., 10032., 10017., 112

In [17]:
# Write the cleaned file to a .csv

health_df.to_csv(path_or_buf='cleaned_data.csv', index=False)


### TODO

---

Each field could use some more cleanup. Specifically, there should be a common value when values are missing, i.e. nan (I believe some fields use "Missing"). The cuisine names could also use some shortening so that they are easier to search for.